# Введение

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

# Подготовка данных

In [1]:
pip install catboost

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from tqdm import tqdm
from nltk.stem import WordNetLemmatizer
import numpy as np
import spacy
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from catboost import CatBoostClassifier

In [3]:
data = pd.read_csv('toxic_comments.csv')

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [5]:
data.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [6]:
data = data.drop(['Unnamed: 0'], axis=1)

In [7]:
# лемматизаия текста
nlp = spacy.load("en_core_web_sm")

def clear(text):
    return " ".join(re.sub(r'[^a-zA-Z]', ' ', text.lower()).split())

def lemmatize(text):
    global nlp
    doc = nlp(text)
    return " ".join([token.lemma_ for token in doc])

data['clear_text'] = data['text'].apply(clear)
data['lemmatized_text'] = data['clear_text'].apply(lemmatize)

**Рассчет TF-IDF**

In [8]:
#разбиение на обучающую, валидационную и тестовую выборки

train, valid = train_test_split(data, test_size=0.2, random_state=12345)
valid, test = train_test_split(valid, test_size=0.5, random_state=12345)

In [9]:
corpus_train = train['lemmatized_text']
corpus_valid = valid['lemmatized_text']
corpus_test = test['lemmatized_text']

In [10]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

count_tf_idf = TfidfVectorizer(stop_words=stopwords)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
#обучающие признаки
tfidf_train = count_tf_idf.fit_transform(corpus_train)
tfidf_valid = count_tf_idf.transform(corpus_valid)
tfidf_test = count_tf_idf.transform(corpus_test)

In [12]:
# целевой признак
target_train = train['toxic']
target_valid = valid['toxic']
target_test = test['toxic']

# Обучение модели

**Логистическая регрессия**

In [13]:
model = LogisticRegression(random_state=12345)
model.fit(tfidf_train, target_train)

C:\Users\Admin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=12345)

In [14]:
predictions_valid = model.predict(tfidf_valid)

In [15]:
f1_score(target_valid, predictions_valid)

0.7476705180767798

**CatBoostClassifier**

In [16]:
best_result = 0.75
best_iter = 0


for i in range(10, 101, 10):
    model = CatBoostClassifier(loss_function='Logloss', iterations=i)

    model.fit(tfidf_train, target_train, verbose=10)
    
    predictions_valid = model.predict(tfidf_valid)
    
    result = f1_score(target_valid, predictions_valid)
    
    if result > best_result:
        best_result = result
        best_iter = i
        
print(best_result)    
print(best_iter)

Learning rate set to 0.5
0:	learn: 0.3416542	total: 2.88s	remaining: 25.9s
9:	learn: 0.1818331	total: 23.1s	remaining: 0us
Learning rate set to 0.5
0:	learn: 0.3416542	total: 2.27s	remaining: 43.2s
10:	learn: 0.1788064	total: 24.8s	remaining: 20.3s
19:	learn: 0.1591385	total: 44.1s	remaining: 0us
Learning rate set to 0.5
0:	learn: 0.3416542	total: 2.28s	remaining: 1m 6s
10:	learn: 0.1788064	total: 24.6s	remaining: 42.4s
20:	learn: 0.1574732	total: 46.1s	remaining: 19.7s
29:	learn: 0.1471153	total: 1m 5s	remaining: 0us
Learning rate set to 0.5
0:	learn: 0.3416542	total: 2.26s	remaining: 1m 28s
10:	learn: 0.1788064	total: 23.9s	remaining: 1m 3s
20:	learn: 0.1574732	total: 45.4s	remaining: 41.1s
30:	learn: 0.1462806	total: 1m 6s	remaining: 19.4s
39:	learn: 0.1388492	total: 1m 26s	remaining: 0us
Learning rate set to 0.5
0:	learn: 0.3416542	total: 2.26s	remaining: 1m 50s
10:	learn: 0.1788064	total: 23.8s	remaining: 1m 24s
20:	learn: 0.1574732	total: 45.1s	remaining: 1m 2s
30:	learn: 0.14628

**Вывод**

По итогу обучения модели на валидационной выборке была выбрана модель CatBoostClassifier.

In [18]:
model = CatBoostClassifier(loss_function='Logloss', iterations=90)

model.fit(tfidf_train, target_train, verbose=10)
    
predictions_test = model.predict(tfidf_test)
    
f1_score(target_test, predictions_test)

Learning rate set to 0.5
0:	learn: 0.3416542	total: 2.49s	remaining: 3m 41s
10:	learn: 0.1788064	total: 24.4s	remaining: 2m 55s
20:	learn: 0.1574732	total: 45.9s	remaining: 2m 30s
30:	learn: 0.1462806	total: 1m 7s	remaining: 2m 7s
40:	learn: 0.1381887	total: 1m 28s	remaining: 1m 45s
50:	learn: 0.1323732	total: 1m 51s	remaining: 1m 25s
60:	learn: 0.1275955	total: 2m 12s	remaining: 1m 2s
70:	learn: 0.1240397	total: 2m 33s	remaining: 41.1s
80:	learn: 0.1206828	total: 2m 54s	remaining: 19.4s
89:	learn: 0.1179761	total: 3m 13s	remaining: 0us


0.7605432451751252

# Вывод

В результате работы над проектом были выполнены следующие задачи:

1) Данные были очищены от ненужных символов, затрудняющих обучение модели;

2) Данные были лемматизированы при помощи библиотеки spacy;

3) Были проанализированы две библиотеки для обучения: логистическая регрессия и CatBoostClassifier;

4) По итогу была выбрана модель CatBoostClassifier с количеством итераций 10, итоговой значение метрики f1 составило 0,76.